In [1]:
spark

Intitializing Scala interpreter ...

Spark Web UI available at http://10.80.102.113:4040
SparkContext available as 'sc' (version = 2.3.1, master = local[*], app id = local-1543851031421)
SparkSession available as 'spark'


res0: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@4f963097


In [2]:
spark.version

res1: String = 2.3.1


In [3]:
val sqlContext = new org.apache.spark.sql.SQLContext(sc)
import sqlContext.implicits._

sqlContext: org.apache.spark.sql.SQLContext = org.apache.spark.sql.SQLContext@2c0b8ca8
import sqlContext.implicits._


## Create a DF that references the Parquet files.

In [8]:
val pqSC = sqlContext.read.parquet("data.parquet")
pqScoreCard10Year.printSchema()


root
 |-- value: array (nullable = true)
 |    |-- element: string (containsNull = true)



pqSC: org.apache.spark.sql.DataFrame = [value: array<string>]


In [10]:
pqSC.show()

+--------------------+
|               value|
+--------------------+
|[UNITID, OPEID, O...|
|[100654, 00100200...|
|[100663, 00105200...|
|[100690, 02503400...|
|[100706, 00105500...|
|[100724, 00100500...|
|[100751, 00105100...|
|[100760, 00100700...|
|[100812, 00100800...|
|[100830, 00831000...|
|[100858, 00100900...|
|[100937, 00101200...|
|[101028, 01218200...|
|[101073, 01055400...|
|[101116, 01303906...|
|[101143, 00101500...|
|[101161, 00106000...|
|[101189, 00100300...|
|[101240, 00101700...|
|[101277, 04187200...|
+--------------------+
only showing top 20 rows



# St Thomas RDD  
This RDD contains all of the St. Thomas results over the years in RDD and dataframe format.

Bummed I have to switch back to SQL for this stuff....

In [13]:
val ust = scoreCard10Year.filter(x => x(0) contains "174914")
// df.filter(df("age") > 21).show()

<console>: 46: error: value update is not a member of org.apache.spark.sql.DataFrame

In [ ]:
ust.take(1)

In [ ]:
ust.count()

In [ ]:
val ustHeader = header2016.union(ust)
// ustHeader.map(_.mkString(",")).coalesce(1).saveAsTextFile("ustHeader")

# St Thomas DF

In [29]:
val ustDF = pqSC.filter(pqSC("UNITID") === "174914").show()


org.apache.spark.sql.AnalysisException:  Cannot resolve column name "UNITID" among (value);

In [19]:
ustDF.count()

res12: Long = 0


# Regional Peers RDD

This RDD filters region 4 (upper midwest/plains), Schools that offer graduate degrees as the highest degree, and a carnegie classification of 4 year, medium size, highly residential. 

In [ ]:
val regionalPeers = scoreCard10Year.filter(x => x(18) == "4").filter(x => x(15) == "4").filter(x => x(25) == "13")

In [ ]:
regionalPeers.count()

### Average SAT score for peers in the region. Ours is 1204

In [ ]:
scoreCard10Year.map(x => x(1899)).map(_.mkString("")).take(1) // I verified index 1899 in all of my load RDDs

In [ ]:
regionalPeers.filter(x => x(1899) == "2016").map(x => x(59)).filter(x => x != "NULL").count()


In [ ]:
regionalPeers.map(x => x(59)).filter(x => x != "NULL").map(_.toInt).mean()


# Create an index of the header 

Really had a tough time figuring out how to get the header loaded as just a string. Loading it from the original source results in type RDD[Array[String]], so I ended up using a CSV file that I outputed from the original input. Ended up just needing a **flatMap** to keep it in the correct format. 

# Minnesota Schools  
This rdd contains all of the Minnesota universities for a statewide comparison

In [ ]:
val mnColleges = scoreCard10Year.filter( x => x(5) contains "MN").filter( x => x(1899) contains "2016")

In [ ]:
val mnColLoc = mnColleges.filter(x => x(1899) contains "2016").filter(x => x(36) != "NULL").map(x => x(3) + "," + x(21) + "," + x(22) + "," + x(36))

In [ ]:
// mnColLoc.take(5).foreach(println) // Only needs to run once.

### Create and export RDD to CSV for mapping 

In [ ]:
val mnColLocHead = Array("Name,Lat,Long,AdmRate")
val mnLocRDD = sc.parallelize(mnColLocHead)
val mnColLocFin = mnLocRDD.union(mnColLoc)
//mnColLocFin.coalesce(1).saveAsTextFile("MnSchools") // only run once